<a href="https://colab.research.google.com/github/AntonioE89/LP_Pyomo/blob/main/LP_CPSE_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Continuous Optimization – linear Programming (LP)

## Problem Statement

$$\min_{\mathbf{x}\in\mathbb{R}^{n_\mathbf{x}}} \quad {\bf c}^T{\bf x}$$
subject to:
$$A\mathbf{x}=\mathbf{b}$$
$$\mathbf{x}^{lb}\leq\mathbf{x}\leq\mathbf{x}^{ub}$$

where $A \in \mathbb{R}^{n_\mathbf{x} \times n_\mathbf{x}}$, $\mathbf{b}\in \mathbb{R}^{n_h}$.

### Linear Programming example

Optimization problem:

$$\min_{x_1,x_2} \quad -x_1-x_2$$
subject to:
$$-x_1+x_2 \geq 0$$
$$x_1 \leq 2, \quad x_2 \leq 3$$
$$x_1 , x_2 \geq 0$$


![picture](https://drive.google.com/uc?export=view&id=1sbYGmH1YfI3Q-QaRIoFGpBwahhKBtuN8)

borrowed from Benoit Chachuat's slides*


# Pyomo

## Importing and installing Pyomo and cbc

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

## Pyomo simple example

In [ ]:
# -- importing libraries -- #
from pyomo.environ import *

# create the model
m = ConcreteModel()

# define variables
m.x = Var([1,2], bounds=(0,3))

# define the bounds
m.bound_x1 = Constraint(expr = m.x[1] <= 2)
# define the constraint
m.g1 = Constraint(expr = - m.x[1] + m.x[2] >= 0)

# definint variables
m.obj = Objective(expr = - m.x[1] - m.x[2], sense=minimize)

# display model
m.pprint()

# call solver
SolverFactory('cbc').solve(m).write()

# display solution
print('\nObjective value = ', m.obj())

print('\nDecision Variables')
print('$x_1$ = ', m.x[1]())
print('$x_2$ = ', m.x[2]())

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     3 : False :  True :  Reals
          2 :     0 :  None :     3 : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : - x[1] - x[2]

2 Constraint Declarations
    bound_x1 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : x[1] :   2.0 :   True
    g1 : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :   0.0 : - x[1] + x[2] :  +Inf :   True

1 Suffix Declarations
    dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
        Key : Value

6 Declarations: dual x_in

### Sensitivity analysis

In [ ]:
# -- importing libraries -- #
from pyomo.environ import *

# create the model
m = ConcreteModel()

# define variables
m.x = Var([1,2], bounds=(-np.inf,np.inf))

# for access to dual solution for constraints
m.dual = Suffix(direction=Suffix.IMPORT)

# define the bounds
m.bound_x1_ub = Constraint(expr = m.x[1] <= 2)
m.bound_x2_ub = Constraint(expr = m.x[2] <= 3)
m.bound_x1_lb = Constraint(expr = m.x[1] >= 0)
m.bound_x2_lb = Constraint(expr = m.x[2] >= 0)
# define the constraint
m.g1 = Constraint(expr = - m.x[1] + m.x[2] >= 0)

# definint variables
m.obj = Objective(expr = - m.x[1] - m.x[2], sense=minimize)

# display model
m.pprint()

# call solver
SolverFactory('cbc').solve(m)

# display solution
print('\nObjective value = ', m.obj())

print('\nDecision Variables')
print('$x_1$ = ', m.x[1]())
print('$x_2$ = ', m.x[2]())

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  -inf :  None :   inf : False :  True :  Reals
          2 :  -inf :  None :   inf : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : - x[1] - x[2]

5 Constraint Declarations
    bound_x1_lb : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   0.0 : x[1] :  +Inf :   True
    bound_x1_ub : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf : x[1] :   2.0 :   True
    bound_x2_lb : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   0.0 : x[2] :  +Inf : 

In [ ]:
print("\nSolution")
print(f"x1 = {m.x[1]()}")
print(f"x2 = {m.x[2]()}")

print("\nSensitivity Analysis")
print(f"x1_ub = {m.dual[m.bound_x1_ub]}")
print(f"x2_ub = {m.dual[m.bound_x2_ub]}")
print(f"x1_lb = {m.dual[m.bound_x1_lb]}")
print(f"x2_ub = {m.dual[m.bound_x2_lb]}")
print(f"g1 = {m.dual[m.g1]}")


Solution
x1 = 2.0
x2 = 3.0

Sensitivity Analysis
x1_ub = -1.0
x2_ub = -1.0
x1_lb = 0.0
x2_ub = 0.0
g1 = 0.0


In [ ]:
Ahat_a = np.array([[1,0],[0,1]])
c      = np.array([[-1,-1]]).T
lam    = c.T@np.linalg.inv(Ahat_a)
print('Lagrange multipliers = ',lam)

Lagrange multipliers =  [[-1. -1.]]


For more information see: https://jckantor.github.io/ND-Pyomo-Cookbook/02.01-Production-Models-with-Linear-Constraints.html
